## Lectura de PDF

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path="Investigación de WindSurf.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [5]:
print(f"{pages[5].metadata}\n")
print(pages[5].page_content)

{'producer': 'Skia/PDF m137 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Investigación de WindSurf', 'source': 'Investigación de WindSurf.pdf', 'total_pages': 9, 'page': 5, 'page_label': '6'}

INFORME
 
WINDSURF
                      
5.  ¿Qué  tipo  de  almacenamiento  usa  y  dónde  se  
almacenan?
 
Su  almacenamiento  es  de  tipo  Vectorial.  Esto  se  debe  a  que  las  incrustaciones  son  
representaciones
 
numéricas
 
de
 
alta
 
dimensión.
  El  lugar  donde  se  almacena  depende  de  su  versión:   En  el  escenario  Local ,  las  incrustaciones  se  guardan  directamente  en  el  dispositivo  del  
usuario
 
bajo
 
cifrado
 
AES-256,
 
mientras
 
que
 
los
 
índices
 
están
 
optimizados
 
para
 
ejecutarse
 
localmente
 
en
 
CPU
 
o
 
GPU.
 
En
 
el
 
escenario
 
Enterprise
,
 
la
 
información
 
se
 
administra
 
en
 
servidores
 
aislados,
 
lo
 
que
 
aporta
 
una
 
capa
 
extra
 
de
 
seguridad
 
y
 
escalabilidad.
 
Además,
 
en
 
ambos


## Búsqueda vectorial sobre pdf

In [43]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")




In [18]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore.from_documents(pages, OpenAIEmbeddings(api_key=api_key))
docs = vector_store.similarity_search("Integra Linters?", k=2)
for doc in docs:
    print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]}\n')

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Page 3: INFORME
 
WINDSURF
                      
puede  ofrecer  respuestas  y  soluciones  más  precisas  y  relevantes,  adaptándose  al  flujo  de  
trabajo
 
del
 
usuario
 
de
 
manera
 
eficiente.
  Análisis :  Una  vez  que  Cascade  recibe  su  entrada,  analiza  tanto  el  código  seleccionado  
c

Page 2: INFORME
 
WINDSURF
                      
1.  Características  claves  
Es  similar  a  Cursor,  pero  con  una  peculiaridad:  utiliza  Flows,  que  combinan  Agentes  y  
Copilots.
 
Al
 
integrar
 
ambas
 
herramientas,
 
se
 
mejora
 
el
 
flujo
 
de
 
desarrollo
 
del
 
programador.
 
Con  esta



# Incrustación de texto

In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key)

In [22]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


(5, 3072)

## Lector de PDFs

### Carga y extracción de texto desde PDFs

In [47]:
from langchain_community.document_loaders import PyMuPDFLoader

# Cargar múltiples PDFs
pdf_paths = ["Investigación de WindSurf.pdf", "nke-10k-2023.pdf"]
docs = []
for path in pdf_paths:
    loader = PyMuPDFLoader(path)
    pages = loader.load()  # pages es lista de Document con page_content y metadata
    docs.extend(pages)


### División de texto en fragmentos (chunking)

In [48]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=150)
chunks = splitter.split_documents(docs)  # Se generan Document separados en trozos


### Generación de embeddings

In [51]:
from langchain_openai import OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = "TU_API_KEY"  # Configura tu API key de OpenAI
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key)


### Almacenar embeddings en FAISS

In [54]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(chunks, embeddings_model)


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: Loading faiss with AVX2 support.
INFO: Successfully loaded faiss with AVX2 support.
INFO: Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


###  Responder consultas vía similitud semántica

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

retriever = db.as_retriever()  # transforma FAISS en un retriever

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(api_key=api_key,temperature=0),        # modelo de lenguaje (e.g. GPT-3.5)
    chain_type="stuff",
    retriever=retriever
)

query = "¿Que es Cascade?"
result = qa_chain.run(query)
print(result)


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


 Cascade es una herramienta de inteligencia artificial que ofrece respuestas y soluciones precisas y relevantes para el flujo de trabajo del usuario. Utiliza reglas y memorias configuradas, selecciona un modelo de IA adecuado, accede a diversas herramientas y genera sugerencias de código, modificaciones o respuestas en función de la solicitud del usuario. También está integrado con linters y puede identificar problemas en el código y ofrecer soluciones.


: 